## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from g_config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database2.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vardo,NO,70.37,31.11,35.60,95,75,26.40,broken clouds
1,1,Cape Town,ZA,-33.93,18.42,62.01,93,75,6.93,broken clouds
2,2,Ponta Do Sol,PT,32.67,-17.10,64.40,68,40,13.87,scattered clouds
3,3,Mabaruma,GY,8.20,-59.78,78.87,82,58,6.13,broken clouds
4,4,Hermanus,ZA,-34.42,19.23,63.00,85,63,2.44,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             203
City                   203
Country                202
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df["Country"].isnull().sum()

1

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

Unnamed: 0             202
City                   202
Country                202
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mabaruma,GY,78.87,broken clouds,8.20,-59.78,
15,Hithadhoo,MV,83.44,broken clouds,-0.60,73.08,
18,Bathsheba,BB,80.60,scattered clouds,13.22,-59.52,
20,The Valley,AI,82.40,few clouds,18.22,-63.06,
30,Srikakulam,IN,84.85,overcast clouds,18.30,83.90,
31,Bambous Virieux,MU,77.00,scattered clouds,-20.34,57.76,
34,Upington,ZA,77.00,broken clouds,-28.45,21.26,
38,Georgetown,MY,86.00,scattered clouds,5.41,100.34,
39,Garissa,KE,76.86,broken clouds,-0.45,39.64,
51,Xapuri,BR,76.17,light rain,-10.65,-68.50,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mabaruma,GY,78.87,broken clouds,8.20,-59.78,
15,Hithadhoo,MV,83.44,broken clouds,-0.60,73.08,Scoop Guest House
18,Bathsheba,BB,80.60,scattered clouds,13.22,-59.52,Atlantis Hotel
20,The Valley,AI,82.40,few clouds,18.22,-63.06,CeBlue Villas & Beach Resort
30,Srikakulam,IN,84.85,overcast clouds,18.30,83.90,Sriram residency


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
15,Hithadhoo,MV,83.44,broken clouds,-0.60,73.08,Scoop Guest House
18,Bathsheba,BB,80.60,scattered clouds,13.22,-59.52,Atlantis Hotel
20,The Valley,AI,82.40,few clouds,18.22,-63.06,CeBlue Villas & Beach Resort
30,Srikakulam,IN,84.85,overcast clouds,18.30,83.90,Sriram residency
31,Bambous Virieux,MU,77.00,scattered clouds,-20.34,57.76,Casa Tia Villa


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))